In [1]:
import warnings
from importlib import reload

with warnings.catch_warnings():
    ## ignore future warnings from too-advanced numpy version
    warnings.simplefilter("ignore")    
    from keras.models import Sequential
    from keras.layers import Dense, Activation, Dropout
import pandas as pd, numpy as np

from sklearn import preprocessing

Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
import energy
from energy import MissingStrategy
_ = reload(energy)

In [3]:
MAX_ROWS = 12*1000*1000
try:
    combined_df = energy.Data.Merged(MAX_ROWS).data
except FileNotFoundError:
    weather = energy.Data.Weather(MAX_ROWS)
    meter = energy.Data.Meter(MAX_ROWS)
    buildings = energy.Data.Buildings()
    combined_df = energy.merge_data(
        weather_df=weather.train, 
        meter_df=meter.train,
        buildings_df=buildings.data)
    

In [4]:
if 'minute' not in combined_df.columns:
    combined_df['timestamp'] = pd.to_datetime(combined_df['timestamp'])
    combined_df['minute'] = combined_df['timestamp'].apply(lambda ts: ts.minute)
    combined_df['second'] = combined_df['timestamp'].apply(lambda ts: ts.second)
    combined_df['week'] = combined_df['timestamp'].apply(lambda ts: ts.week)
    combined_df['day'] = combined_df['timestamp'].apply(lambda ts: ts.day)

In [7]:
# combined_df.to_csv(energy.Data.Merged.train_file, index=False)

In [6]:
combined_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,minute,second,week,day
0,0,0,2016-01-01 00:00:00,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,0,0,53,1
1,1,0,2016-01-01 00:00:00,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,0,0,53,1
2,2,0,2016-01-01 00:00:00,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,0,0,53,1
3,3,0,2016-01-01 00:00:00,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,0,0,53,1
4,4,0,2016-01-01 00:00:00,0.0,0,Education,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,0,0,53,1


In [5]:
time_cols = ['second', 'minute', 'day', 'week']
weather_cols = [
    'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 
    'sea_level_pressure', 'wind_direction', 'wind_speed']
X_df = combined_df[[
    'square_feet', 'meter', 'year_built', 'primary_use'] +
    weather_cols +
    time_cols]
y = combined_df[['meter_reading']].values
del(combined_df)

le = preprocessing.LabelEncoder()
le.fit(list(X_df.loc[:, 'primary_use'].unique()))
X_df.loc[:, 'primary_use'] = le.transform(X_df['primary_use'])


In [6]:
X = X_df.values
del(X_df)
assert not np.isnan(y).any()

In [7]:
missing_strategy = energy.MissingStrategy.USE_COLUMN_MEAN
X, y = energy.handle_missing_predictor_values(X, y, missing_strategy)
assert not np.isnan(X).any()

In [8]:
X_scaled = preprocessing.scale(X)
del(X)

In [13]:
model = Sequential()
def add_relu_with_scaled_dropout(dim):
    model.add(Dense(dim, activation='relu'))
    model.add(Dropout(1 / dim))

model.add(Dense(12, activation='relu', input_dim=X_scaled.shape[1]))
model.add(Dropout(1/12))
for dim in (10, 8, 6):
    add_relu_with_scaled_dropout(dim)
model.add(Dense(4, activation='relu'))
model.add(Dense(2, activation='relu'))
model.add(Dense(1, activation='relu'))

In [15]:
rmsle = energy.define_rmsle(X_scaled.shape[0])
model.compile(optimizer='Nadam',
              loss='mean_squared_logarithmic_error', metrics=['mse', rmsle])

In [16]:
model.fit(X_scaled, y, epochs=3, batch_size=10000)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/3
12000000/12000000 [==============================] - 33s 3us/step - loss: 4.7435 - mse: 38522474496.0000 - rmsle: 0.0616
Epoch 2/3
12000000/12000000 [==============================] - 33s 3us/step - loss: 3.6902 - mse: 38521724928.0000 - rmsle: 0.0554
Epoch 3/3
12000000/12000000 [==============================] - 32s 3us/step - loss: 3.4515 - mse: 38521622528.0000 - rmsle: 0.0536
